<img src="./images/DLI_Header.png" width=400/>

# Reconstructing Outdoor Environments for Physical AI Simulation with 3D Gaussian Splatting in NVIDIA Isaac Sim

**Table of Contents**
<br>
In this notebook we demonstrate how to reconstruct an outdoor scene. This includes the following sections:

[Part 1: Reconstructing a Scene with Gaussian Splatting](#part-1-reconstructing-a-scene-with-gaussian-splatting)<br>
    &emsp;&emsp;1. [Run Structure from Motion (COLMAP)](#run-structure-from-motion-colmap)<br>
    &emsp;&emsp;2. [fVDB Reality Capture](#fvdb-reality-capture)<br>
    &emsp;&emsp;&emsp;&emsp;a) [Setup and Imports](#setup-and-imports) <br>
    &emsp;&emsp;&emsp;&emsp;b) [Preprocess Scene](#preprocess-scene) <br>
    &emsp;&emsp;&emsp;&emsp;c) [Train a Gaussian Splatting Scene](#train-a-gaussian-splatting-scene) <br>
    &emsp;&emsp;&emsp;&emsp;d) [Convert to a Mesh](#convert-to-a-mesh) <br>
    &emsp;&emsp;&emsp;&emsp;e) [Create Isaac Sim Ready Files](#create-isaac-sim-ready-files)<br>


## Overview
In this lab we will learn to how to reconstruct an outdoor scene to test robots using NVIDIA fVDB framework and NVIDIA Omniverse NuRec rendering in Isaac Sim. We will walk through core reconstruction and rendering technologies, with a step-by-step workflow for simulating an entire outdoor environment for testing any robot. We will learn how to position images using structure from motion, train a 3D Gaussian splatting scene, extract 3D mesh, and convert to USD for simulation in Isaac Sim. This lab is split into two parts. In [Part 1](#part-1-creating-a-digital-twin-with-gaussian-splatting), we will learn how to gather good data, use a SfM tool, and train a Gaussian splatting scene. In [Part 2](#Part-2-creating-an-isaac-sim-scene), we will switch focus to robotic simulation. We will import the files from [Part 1](#part-1-creating-a-digital-twin-with-gaussian-splatting) into Isaac Sim and use Isaac Lab to move a [Spot](https://bostondynamics.com/products/spot/) robot around the scene.

## Part 1: Reconstructing a Scene with Gaussian Splatting

### Run Structure from Motion (COLMAP)
Many radiance field rendering methods, including 3D Gaussian Splatting, require the camera positions and a sparse point cloud of the scene for initialization. We currently have a folder of raw images, but no corresponding camera location or pose information. We can use a structure from motion tool (SfM) to estimate where the camera was for each image and to create a sparse point cloud of the scene. A commonly used SfM tool is [COLMAP](https://colmap.github.io/install.html), which we will use in combination with GLOMAP. [GLOMAP](https://github.com/colmap/glomap) replaces COLMAP's mapper step, focusing on global positioning rather than incremental, and can run 10 or even 100 times faster than COLMAP's. Below we have provided example commands. There's no need to run them here as we have provided the result from an existing SfM run. You can run these commands on your own data by downloading COLMAP and GLOMAP and changing the **/Data/Path** to a directory that contains the **images_raw** folder of your data.


In [ ]:
# Example COLMAP & GLOMAP commands

# # Run the feature extract to identify key points
# !colmap feature_extractor \
#     --database_path /Data/Path/database.db \
#     --image_path /Data/Path/images_raw \
#     --ImageReader.camera_model PINHOLE \
#     --ImageReader.single_camera 1 \
#     --SiftExtraction.use_gpu 1 \
#     --SiftExtraction.max_image_size 7096 \
#     --SiftExtraction.max_num_features 20000 \
#     --SiftExtraction.num_threads 14

# # Mature the features across images
# !colmap exhaustive_matcher \
#     --database_path /Data/Path/database.db \
#     --SiftMatching.use_gpu 1 \
#     --SiftMatching.max_num_matches 60000 \
#     --SiftMatching.guided_matching=true

# # Create a sparse 3D point cloud of the scene using GLOMAP's global mapper
# !glomap mapper \
#     --database_path /Data/Path/database.db \
#     --image_path /Data/Path/images_raw \
#     --output_path /Data/Path/sparse \
#     --GlobalPositioning.use_gpu 1 \
#     --BundleAdjustment.use_gpu 1

# # Align 3d model by applying transformations
# !colmap model_aligner \
#     --input_path /Data/Path/sparse/0 \
#     --output_path /Data/Path/sparse/aligned \
#     --database_path /Data/Path/database.db \
#     --ref_is_gps 1 \
#     --alignment_type ECEF \
#     --alignment_max_error 3.0

# # Undistort original input images so they are as if they were taken with a pinhole camera
# !colmap image_undistorter \
#     --image_path /Data/Path/images_raw \
#     --input_path /Data/Path/sparse/0 \
#     --output_path /Data/Path \
#     --output_type=COLMAP

Let's take a closer look at the files from the provided SfM run.

In [ ]:
import logging

import cv2
import fvdb
import matplotlib.pyplot as plt
import numpy as np
import torch

import fvdb_reality_capture as frc

# Let's use verbose logging to track what happens under the hood.
# For less output set level=logging.WARN. For more set level=logging.DEBUG
logging.basicConfig(level=logging.INFO, format="%(levelname)s : %(message)s")


In [ ]:
# Download example data for running splats on
frc.tools.download_example_data(dataset="safety_park", download_path="./data")

In [ ]:
# Print contents of directory in tree like format
!tree ./data/safety_park

Now let's delve deeper into each of these files and folders.

**images_raw**: Contains the original images <br>
**images**: Contains undistorted images <br>
**sparse**: Contains the sparse 3D reconstruction of the scene in folder 0. If COLMAP cannot register the images into 1 single scene, it will be split in to additional numbered folders. <br>
**cameras.bin**: Camera intrinsics including camera IDs, camera models, and sensor dimensions. <br>
**images.bin**: Camera poses and keypoints for all reconstructed images. <br>
**points3D.bin**: The sparse 3D point cloud <br>
To learn more these files COLMAP produces see the [COLMAP's Output Format page](https://colmap.github.io/format.html).


### fVDB Reality Capture
Now that we have a SfM run we can visualize it and use it to train a Gaussian splatting scene. We will use an [fVDB](https://github.com/openvdb/fvdb-core) example project called [fVDB Reality Capture](https://github.com/openvdb/fvdb-reality-capture) for visualization, manipulation and training. fVDB is a framework for encoding and operating on sparse voxel hierarchies of features in PyTorch. Voxels are like pixels but they are cubes instead of squares, making them three dimensional. Sparse means we only have voxels in areas of our scene that are occupied, voxels that don't contain anything are not represented. fVDB Reality Capture (fRC) is toolbox for reality capture tasks built on top of fVDB. It gathers the tools required to create an Isaac Sim compatible 3D reconstruction from a set of images and uses fVDB to make the tools fast and efficient.



#### Setup and Imports

Before we get started, let's import the packages we need. Here's an overview of some of the important ones:

* `logging`
    - We'll use the built-in Python `logging` module, and call `logging.basicConfig()` which will cause functions within `fvdb_reality_capture` to log to stdout. You don't have to enable this, but it's useful to see what's happening under the hood.
* `fvdb` 
    - We use `fvdb` for the underlying Gaussian splat data structure (`fvdb.GaussianSplat3d`) which provides fast and scalable rendering, and for interactive visualization (using the `fvdb.viz`) module.
* `fvdb_reality_capture` 
    - We use this for core algorithms that reconstruct scenes from sensors help us read and process capture data


#### Visualize a Sfm Scene

We can use fRC to view and manipulate our SfM scene since it supports loading in capture data stored in different formats into a common representation that can be easily manipulated by users. To do this, data from a capture is stored in an `fvdb_reality_capture.SfmScene` object which acts as an in-memory representation of a 3D capture. We will use this object to manipulate and visualize our SfM scene. Let's load our scene into a `SfmScene` by passing in the folder containing all the SfM output and images.

In [ ]:
# Load Safety Park SfM Scene
sfm_scene = frc.sfm_scene.SfmScene.from_colmap("./data/safety_park")

Let's take a closer look at what a SfmScene scene consists of.
* `SfmScene.cameras`: A dictionary mapping unique camera IDs to `SfmCameraMetadata` objects which contain camera parameters (e.g. projection matrices, distortion parameters). The size of this dictionary matches the number of cameras used to capture the scene (so if you scanned a scene with a pair of stereo cameras, then len(SfmScene.cameras) will be 2).
* `SfmScene.images`: A list of SfmImageMetadata objects which contain paths to the images and optional masks, a reference to the camera (SfmCameraMetadata) used to capture each image, their camera-to-world (and inverse) transformations, and the set of 3D points visible in each image.
* `points/points_rgb/points_err`: Numpy arrays of shape (N,3)/(N,3)/(N,) encoding known surface points in the scene, their RGB colors, and an unnormalized confidence value of the accuracy of that point. Note, N denotes the number of points here. <br>

Now that we have a loaded SfmScene, let's plot some of its images, and the projected 3D points within those images. We can use this object to display our training images and project the 3D points from the sparse point cloud on to them.  We'll visualize the 3D points and cameras interactively using `fvdb.viz` and we'll plot some images with their visible points using `matplotlib`. The `fvdb.viz` module provides a high performance debug visualizer written in [vulkan](https://www.vulkan.org/). It spins up a small HTTP server which streams a visualization to a notebook or a browser.

In [ ]:
# Visualize an image in an SfmScene and the 3D points visible from that images
# projected onto the image plane as blue dots.
def plot_image_from_scene(scene: frc.sfm_scene.SfmScene, image_id: int):
    image_meta: frc.sfm_scene.SfmPosedImageMetadata = scene.images[image_id]
    camera_meta: frc.sfm_scene.SfmCameraMetadata = image_meta.camera_metadata

    # Get the visible 3d points for this image
    visible_points_3d: np.ndarray = scene.points[image_meta.point_indices]

    # Project those points onto the image plane
    # 1. Get the world -> camera space transform and projection matrix
    world_to_cam_matrix: np.ndarray = image_meta.world_to_camera_matrix
    projection_matrix: np.ndarray = camera_meta.projection_matrix
    # 2. Transform world points to camera space
    visible_points_3d_cam_space = world_to_cam_matrix[:3,:3] @ visible_points_3d.T + world_to_cam_matrix[:3,3:4]
    # 3. Transform camera space coordinates to image space
    visible_points_2d = projection_matrix @ visible_points_3d_cam_space
    visible_points_2d /= visible_points_2d[2]

    # Load the image and convert to RGB (OpenCV uses BGR by default)
    loaded_image = cv2.imread(image_meta.image_path)
    assert loaded_image is not None, f"Failed to load image at {image_meta.image_path}"
    loaded_image = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2RGB)

    # Plot the image and projected points
    plt.title(f"SfmScene Image {image_id}")
    plt.axis("off")
    plt.imshow(loaded_image)
    plt.scatter(visible_points_2d[0], visible_points_2d[1], color="#432de9", marker=".", s=2)


# Plot three images from the scene and their visible 3D points alongside each other
plt.figure(figsize=(25, 4.25))
plt.subplot(1, 3, 1)
plot_image_from_scene(sfm_scene, 8)
plt.subplot(1, 3, 2)
plot_image_from_scene(sfm_scene, 16)
plt.subplot(1, 3, 3)
plot_image_from_scene(sfm_scene, 32)
plt.show()


You can interact with the above viewer running on [localhost:8000](http://localhost:8000/). For zooming, hold the scroll wheel and drag the mouse up or down. Zoom out enough to see the camera positions represented in green, You can also pan with the right mouse button and rotate with the left mouse button. 

If you want to know more about manipulation of SfmScenes using fVDB see the [Reconstructing a Gaussian Splat Radiance Field and High Quality Mesh from a Capture
](https://github.com/openvdb/fvdb-reality-capture/blob/main/notebooks/radiance_field_and_mesh_reconstruction.ipynb) notebook.

#### Preprocess Scene
You may have noticed the scene in the viewer is rotated with respect to the canonical axes. It's also very noisy with a lot of outlier points.
This is because the structure-from-motion algorithm which produced this data had a lot of noisy predictions, and predicted points and cameras in a rotated coordinate frame.
Before we reconstruct a Gaussian Splat radiance field, let's clean up our input data a bit. We'll apply the following steps:

 1. Downsample the images by a factor of 2 to speed up Gaussian Splat optimization (loading big images can be time consuming), 
 2. Normalize the scene to an east-north-up coordinate system.
 3. Remove outlier points below the bottom 3rd and above top 97th percentiles along the X, Y, and Z, axis.
 4. Remove any images with fewer than 50 visible points (these images are likely to have bad pose estimates)

`fvdb_reality_capture` makes this kind of cleanup easy, efficient, and seamless using the `transforms` module. 
Transforms are classes which define a transformation of an `SfmScene`. They inherit from `fvdb_reality_capture.BaseTransform`, and their `__call__` operator accepts an `SfmScene` as input and produces a new `SfmScene` as output. Let's look at some code and visualizations and then dive into how this works.

In [ ]:
# Apply preprocessing using transforms
import fvdb_reality_capture.transforms as fvtransforms
cleanup_and_resize_transform = fvtransforms.Compose(
    # Downsample images by factor of 2
    fvtransforms.DownsampleImages(image_downsample_factor=2, image_type="jpg", rescaled_jpeg_quality=95),
    # Normalize the scene to an est-north-up coordinate system.
    fvtransforms.NormalizeScene(normalization_type="ecef2enu"),
    # Remove outlier points
    fvtransforms.PercentileFilterPoints(percentile_min=3.0, percentile_max=97.0),
    # filters outlier points, and removes images with too few points.
    fvtransforms.FilterImagesWithLowPoints(min_num_points=50),
)

# Apply the transforms
sfm_scene = cleanup_and_resize_transform(sfm_scene)

#### Train a Gaussian Splatting Scene 
In this section we will walk through training a Gaussian splatting scene from a set of images and a SfM output, but let's start with what a Gaussian splatting scene is.

[3D Gaussian Splatting](https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/) is the dominant radiance field representation. A [radiance field](https://radiancefields.com/) is a 3D scene representation that specifies the color of a point along a given view direction. Radiance fields enable high fidelity visualization of 3D captures with realistic lighting effects. 3D Gaussian splats encode a radiance field function as a sum of 3D Gaussian functions or splats, parameterized by their means (positions), rotations, and scales. Additionally each splat has an opacity value and a set of spherical harmonics that map the direction in which a Gaussian is viewed to a color. Each scene is made up of thousands to millions of these splats. Think of them as similar to voxels, or points in a point cloud. We start with splats in the same positions as the points in the sparse point cloud from our SfM run. During training splats are added, removed, split, and changed until the Gaussian splatting scene is capable of producing renders that are nearly identical to the training images. 
<div>
<img src="https://fvdb-data.s3.us-east-2.amazonaws.com/fvdb-reality-capture/doc_figures/fvdb_gs_opt.jpg" width="80%"/>
</div>

Now that we have an understanding of what a Gaussian splatting scene is, let's train our own. The core API for Gaussian Splat reconstruction is the `fvdb_reality_capture.GaussianSplatReconstruction` class. It accepts an input `SfmScene` and optional config parameters, and produces an `fvdb.GaussianSplat3d` reconstructing the scene. This functionally is also written in `/fvdb_reality_capture/cli/frgs/_reconstruct.py`. We will show how to use both methods.

In [ ]:
# Create config so we can edit various parameters
cfg = frc.radiance_fields.GaussianSplatReconstructionConfig()
cfg.eval_at_percent = [100]  # save evaluation for the end of training
cfg.save_at_percent = [100]  # Don't save the model until the end of training
cfg.refine_every_epoch = 4.5  # How often to refine Gaussians during optimization
cfg.pose_opt_start_epoch = 100  # Epoch at which we start optimizing camera positions

# Create a GaussianSplatReconstruction runner
runner = frc.radiance_fields.GaussianSplatReconstruction.from_sfm_scene(
    config=cfg,  # The reconstruction config
    sfm_scene=sfm_scene,  # The SfM scene
    use_every_n_as_val=-1,  # Don't leave any images for validation
)

# Train
runner.optimize()

# Save results
runner.save_ply("reconstruction.ply")

Higher resolution images and a larger quantity of images will cause training to take longer.

Our optimizer was created with several different parameters. They effect when we save the model, how often the splats are refined, and more. There are many more parameters you can vary for training beyond these few. We will use the command-line interface (CLI) tool called `frgs` for easy usage of Gaussian splatting tools. To learn more about `frgs` please see [Reconstruction on the CLI with frgs
](https://openvdb.github.io/fvdb-reality-capture/tutorials/frgs.html) documentation. Now we'll run the reconstruct script with the help flag to see all the possible arguments and parameters.

In [ ]:
!frgs reconstruct --help

Trying changing and adding parameters to see how they effect training. To stop the run early you can click the square button at the top of the notebook to interrupt the kernel.

#### Convert to a Mesh
In order for a robot to move around in Isaac Sim there needs to be a surface it can interact with. Gaussian splatting scenes cannot represent a solid surface in Isaac Sim, so we need to convert our splats to a mesh capable of acting as a collider. `fvdb_reality_capture.tools` provides `mesh_from_splats_dlnr` which will use a foundation model to compute high quality depth maps from images rendered from our Gaussian splat. 
The method works by rendering stereo pairs from the splat scene, and uses the [DLNR](https://openaccess.thecvf.com/content/CVPR2023/papers/Zhao_High-Frequency_Stereo_Matching_Network_CVPR_2023_paper.pdf) foundation model to perform stereo depth estimation. The DLNR model is a high-frequency stereo matching network that computes optical flow and disparity maps between two images.
The we can fuse the depth estimations into a truncated signed distance field (TSDF) using the [TSDF](https://www.microsoft.com/en-us/research/publication/kinectfusion-real-time-3d-reconstruction-and-interaction-using-a-moving-depth-camera/) fusion algorithm. TSDF fusion accumulates noisy depth maps into a voxel grid, to approximate a signed distance field near the surface of the object. `fvdb_reality_capture` makes use of `fvdb-core` to provide a high performance implementation of TSDF integration on sparse voxels. A mesh can then be extracted from the TSDF using the marching cubes algorithm implemented in `fvdb.marching_cubes.marching_cubes`. This allows us to generate meshes from Gaussian splats at very high resolutions on the GPU while using minimal memory.


In [ ]:
from fvdb_reality_capture.tools import mesh_from_splats
import point_cloud_utils as pcu

# The truncation margin determines the width of the narrow band around the surface in which we compute the TSDF.
# A larger margin will produce coarser voxels, while a smaller margin will produce finer voxels but may miss some surface details.
# Here we pick a truncation margin of 0.25 world units in our scene.
truncation_margin = 0.5

# load the Gaussian splatting scene
pretrained_splat_path = "reconstruction.ply"
model, metadata = fvdb.GaussianSplat3d.from_ply(pretrained_splat_path, device="cuda")
# This function returns a tensor of vertices, faces, and colors for the mesh.
# The vertices have shape (num_vertices, 3), the faces have shape (num_faces, 3),
# and the colors have shape (num_vertices, 3). The colors are in the range [0, 1].
v, f, c = mesh_from_splats(model, sfm_scene.camera_to_world_matrices, sfm_scene.projection_matrices, sfm_scene.image_sizes, truncation_margin)

# Save the mesh as a PLY file for viewing in external tools using point_cloud_utils (https://fwilliams.info/point-cloud-utils/) [3]
pcu.save_mesh_vfc("safety_park_mesh.ply", v.cpu().numpy(), f.cpu().numpy(), c.cpu().numpy())

print(f"Reconstructed mesh with {v.shape[0]:,} vertices and {f.shape[0]:,} faces")

#### Create Isaac Sim Ready Files

As we saw in viewer, splats end up beyond the main focus of our scene. The training process tries to reconstruct the all the details in the images, even details far in the horizon. This can lead to messy looking edges. We will crop both the mesh and splats to focus on the center of our scene using **create_isaac_sim_ready_files.py**. There are also a few other things this script does to make our mesh and splats compatible with Isaac Sim.

* Coordinate System: Isaac Sim uses different world axes coordinates than COLMAP, it assumes Z+ is up rather than -Y, so we will rotate the scene accordingly. 
* Water Tight: To ensure our mesh will act as a collider in Isaac Sim we will make it water tight. This will fill in holes in the mesh and smooth it out.
* Format Mesh: We will convert the mesh from a PLY to an OBJ, a file type compatible Isaac Sim.
* Format Splats: Isaac Sim doesn't understand a Gaussian splatting scene in PLY format, so we will have to convert ours into something Isaac Sim can render. USDs are commonly used with Isaac Sim, so we will convert our scene to a Universal Scene Description Zip (USDZ), a compressed version of USD.

In [ ]:
# Run the script to make Isaac Sim Ready Files
# --input_splat: Location of Gaussian splatting scene
# --input_mesh: Location of mesh (PLY)
# --output_path: Where to save the OBJ and USDZ (no file extension)
# --bbox: Box to crop to
# --resolution: How detailed our mesh will be. Increase if you want more faces and vertices.
!python ../scripts/create_isaac_ready_files.py \
--input-splat reconstruction.ply \
--input-mesh safety_park_mesh.ply \
--output-path safety_park_cropped \
--bbox -100 -70 -20 110 90 20 \
--resolution 10000
